In [1]:
from pathlib import Path

import pandas as pd
from lightning import pytorch as pl
import ray
from ray import tune
from ray.train import CheckpointConfig, RunConfig, ScalingConfig
from ray.train.lightning import (RayDDPStrategy, RayLightningEnvironment,
                                 RayTrainReportCallback, prepare_trainer)
from ray.train.torch import TorchTrainer
from ray.tune.search.hyperopt import HyperOptSearch
from ray.tune.search.optuna import OptunaSearch
from ray.tune.schedulers import FIFOScheduler

from chemprop import data, featurizers, models, nn

/home/aih/serra.korkmaz/miniconda3/envs/saturn/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-11 23:37:40,280	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-03-11 23:37:40,551	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-03-11 23:37:40,916	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
chemprop_dir = Path.cwd().parent
train_path = "/home/aih/serra.korkmaz/projects/saturn/SurogateModel/training_data_4sm_zinc/train_data_braf.csv"
test_path = "/home/aih/serra.korkmaz/projects/saturn/SurogateModel/training_data_4sm_zinc/test_data_braf.csv"
num_workers = 0 # number of workers for dataloader. 0 means using main process for data loading
smiles_column = 'smiles' # name of the column containing SMILES strings
target_columns = ['value'] # list of names of the columns containing targets

hpopt_save_dir = Path.cwd() / "hpopt_braf_zinc" # directory to save hyperopt results
hpopt_save_dir.mkdir(exist_ok=True)

In [3]:
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)

In [4]:
# Extract SMILES and target values
train_smis = df_train[smiles_column].values
train_ys = df_train[target_columns].values

test_smis = df_test[smiles_column].values
test_ys = df_test[target_columns].values

# Convert data to MoleculeDatapoint format
train_data = [data.MoleculeDatapoint.from_smi(smi, y) for smi, y in zip(train_smis, train_ys)]
test_data = [data.MoleculeDatapoint.from_smi(smi, y) for smi, y in zip(test_smis, test_ys)]


In [5]:
# Initialize featurizer
featurizer = featurizers.SimpleMoleculeMolGraphFeaturizer()

# Create training dataset
train_dset = data.MoleculeDataset(train_data, featurizer)
scaler = train_dset.normalize_targets()

# Split validation set from training data (80% train, 20% validation)
split_idx = int(len(train_data) * 0.8)
val_data = train_data[split_idx:]
train_data = train_data[:split_idx]

# Create validation and test datasets
val_dset = data.MoleculeDataset(val_data, featurizer)
val_dset.normalize_targets(scaler)

test_dset = data.MoleculeDataset(test_data, featurizer)

In [6]:
def train_model(config, train_dset, val_dset, num_workers, scaler):

    # config is a dictionary containing hyperparameters used for the trial
    depth = int(config["depth"])
    ffn_hidden_dim = int(config["ffn_hidden_dim"])
    ffn_num_layers = int(config["ffn_num_layers"])
    message_hidden_dim = int(config["message_hidden_dim"])

    train_loader = data.build_dataloader(train_dset, num_workers=num_workers, shuffle=True)
    val_loader = data.build_dataloader(val_dset, num_workers=num_workers, shuffle=False)

    mp = nn.BondMessagePassing(d_h=message_hidden_dim, depth=depth)
    agg = nn.MeanAggregation()
    output_transform = nn.UnscaleTransform.from_standard_scaler(scaler)
    ffn = nn.RegressionFFN(output_transform=output_transform, input_dim=message_hidden_dim, hidden_dim=ffn_hidden_dim, n_layers=ffn_num_layers)
    batch_norm = True
    metric_list = [nn.metrics.RMSE(), nn.metrics.MAE()]
    model = models.MPNN(mp, agg, ffn, batch_norm, metric_list)

    trainer = pl.Trainer(
        accelerator="auto",
        devices=1,
        max_epochs=20, # number of epochs to train for
        # below are needed for Ray and Lightning integration
        strategy=RayDDPStrategy(),
        callbacks=[RayTrainReportCallback()],
        plugins=[RayLightningEnvironment()],
    )

    trainer = prepare_trainer(trainer)
    trainer.fit(model, train_loader, val_loader)

In [7]:
search_space = {
    "depth": tune.qrandint(lower=2, upper=6, q=1),
    "ffn_hidden_dim": tune.qrandint(lower=300, upper=2400, q=100),
    "ffn_num_layers": tune.qrandint(lower=1, upper=3, q=1),
    "message_hidden_dim": tune.qrandint(lower=300, upper=2400, q=100),
}

In [8]:
ray.init()

scheduler = FIFOScheduler()

# Scaling config controls the resources used by Ray
scaling_config = ScalingConfig(
    num_workers=1,
    use_gpu=False, # change to True if you want to use GPU
)

# Checkpoint config controls the checkpointing behavior of Ray
checkpoint_config = CheckpointConfig(
    num_to_keep=1, # number of checkpoints to keep
    checkpoint_score_attribute="val_loss", # Save the checkpoint based on this metric
    checkpoint_score_order="min", # Save the checkpoint with the lowest metric value
)

run_config = RunConfig(
    checkpoint_config=checkpoint_config,
    storage_path=hpopt_save_dir / "ray_results", # directory to save the results
)

ray_trainer = TorchTrainer(
    lambda config: train_model(
        config, train_dset, val_dset, num_workers, scaler
    ),
    scaling_config=scaling_config,
    run_config=run_config,
)

search_alg = HyperOptSearch(
    n_initial_points=1, # number of random evaluations before tree parzen estimators
    random_state_seed=42,
)

# OptunaSearch is another search algorithm that can be used
# search_alg = OptunaSearch()

tune_config = tune.TuneConfig(
    metric="val_loss",
    mode="min",
    num_samples=2, # number of trials to run
    scheduler=scheduler,
    search_alg=search_alg,
    trial_dirname_creator=lambda trial: str(trial.trial_id), # shorten filepaths

)

tuner = tune.Tuner(
    ray_trainer,
    param_space={
        "train_loop_config": search_space,
    },
    tune_config=tune_config,
)

# Start the hyperparameter search
results = tuner.fit()

(RayTrainWorker pid=28563) Setting up process group for: env:// [rank=0, world_size=1]
(TorchTrainer pid=28295) Started distributed worker processes: 
(TorchTrainer pid=28295) - (node_id=59ac841c515528e1193ca800fd265e608c3742206adb41f7716efabd, ip=10.233.0.20, pid=28563) world_rank=0, local_rank=0, node_rank=0
(RayTrainWorker pid=28563) /home/aih/serra.korkmaz/miniconda3/envs/saturn/lib/python3.11/site-packages/ray/train/lightning/_lightning_utils.py:262: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
(RayTrainWorker pid=28563) `get_trial_name` is deprecated because the concept of a `Trial` will soon be removed in Ray Train.Ray Train will no longer assume that it's running within a Ray Tune `Trial` in the future. See this issue for more context and migration options: https://github.com/ray-project/ray/issues/49454. Disable these warnings by setting the environment vari

(raylet) A worker died or was killed while executing a task by an unexpected system error. To troubleshoot the problem, check the logs for the dead worker. RayTask ID: ffffffffffffffffdff9fb9e8a2e7281adf3748201000000 Worker ID: 5ab7ca9fadc39bf72057d058b322a06b85cfe1b81046c6814d21818a Node ID: 59ac841c515528e1193ca800fd265e608c3742206adb41f7716efabd Worker IP address: 10.233.0.20 Worker port: 44471 Worker PID: 28563 Worker exit type: SYSTEM_ERROR Worker exit detail: Worker unexpectedly exits with a connection error code 2. End of file. There are some potential root causes. (1) The process is killed by SIGKILL by OOM killer due to high memory usage. (2) ray stop --force is called. (3) The worker is crashed unexpectedly due to SIGSEGV or other unexpected errors.


(TorchTrainer pid=28295) Worker 0 has failed.
2025-03-11 23:39:17,570	ERROR tune_controller.py:1331 -- Trial task failed for trial TorchTrainer_62f3f280
Traceback (most recent call last):
  File "/home/aih/serra.korkmaz/miniconda3/envs/saturn/lib/python3.11/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
             ^^^^^^^^^^^^^^^
  File "/home/aih/serra.korkmaz/miniconda3/envs/saturn/lib/python3.11/site-packages/ray/_private/auto_init_hook.py", line 21, in auto_init_wrapper
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/home/aih/serra.korkmaz/miniconda3/envs/saturn/lib/python3.11/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/aih/serra.korkmaz/miniconda3/envs/saturn/lib/python3.11/site-packages/ray/_private/worker.py", line 2771, in get
    values, debugger_breakpoint = worker.get_objects(

Sanity Checking: |          | 0/? [00:00<?, ?it/s]
Epoch 0: 100%|██████████| 28/28 [00:07<00:00,  3.89it/s, v_num=3.41e+7, train_loss_step=1.090]
Validation: |          | 0/? [00:00<?, ?it/s]
(RayTrainWorker pid=29191) 
Validation DataLoader 0:   0%|          | 0/6 [00:00<?, ?it/s]
(RayTrainWorker pid=29191) 
Validation DataLoader 0:  17%|█▋        | 1/6 [00:00<00:00,  7.91it/s]
(RayTrainWorker pid=29191) 
Validation DataLoader 0:  33%|███▎      | 2/6 [00:00<00:00,  6.48it/s]
(RayTrainWorker pid=29191) 
Validation DataLoader 0:  50%|█████     | 3/6 [00:00<00:00,  6.22it/s]
(RayTrainWorker pid=29191) 
Validation DataLoader 0:  67%|██████▋   | 4/6 [00:00<00:00,  5.79it/s]
(RayTrainWorker pid=29191) 
Epoch 0: 100%|██████████| 28/28 [00:09<00:00,  3.09it/s, v_num=3.41e+7, train_loss_step=1.090, val_loss=0.847, train_loss_epoch=0.598]


(RayTrainWorker pid=29191) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/ictstr01/home/aih/serra.korkmaz/projects/saturn/hopt/hpopt_braf_zinc/ray_results/TorchTrainer_2025-03-11_23-38-37/1f3ddaf5/checkpoint_000000)


...

(RayTrainWorker pid=29191) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/ictstr01/home/aih/serra.korkmaz/projects/saturn/hopt/hpopt_braf_zinc/ray_results/TorchTrainer_2025-03-11_23-38-37/1f3ddaf5/checkpoint_000018)


Epoch 19: 100%|██████████| 28/28 [00:06<00:00,  4.20it/s, v_num=3.41e+7, train_loss_step=0.147, val_loss=0.145, train_loss_epoch=0.155]
Validation: |          | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:   0%|          | 0/6 [00:00<?, ?it/s]
(RayTrainWorker pid=29191) 
Validation DataLoader 0:  17%|█▋        | 1/6 [00:00<00:00,  7.50it/s]
(RayTrainWorker pid=29191) 
Validation DataLoader 0:  33%|███▎      | 2/6 [00:00<00:00,  6.50it/s]
(RayTrainWorker pid=29191) 
Validation DataLoader 0:  50%|█████     | 3/6 [00:00<00:00,  6.16it/s]
(RayTrainWorker pid=29191) 
Validation DataLoader 0:  67%|██████▋   | 4/6 [00:00<00:00,  6.02it/s]
(RayTrainWorker pid=29191) 
Validation DataLoader 0:  83%|████████▎ | 5/6 [00:00<00:00,  6.02it/s]
(RayTrainWorker pid=29191) 
Epoch 19: 100%|██████████| 28/28 [00:09<00:00,  3.00it/s, v_num=3.41e+7, train_loss_step=0.147, val_loss=0.145, train_loss_epoch=0.144]


(RayTrainWorker pid=29191) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/ictstr01/home/aih/serra.korkmaz/projects/saturn/hopt/hpopt_braf_zinc/ray_results/TorchTrainer_2025-03-11_23-38-37/1f3ddaf5/checkpoint_000019)


Epoch 19: 100%|██████████| 28/28 [00:09<00:00,  2.92it/s, v_num=3.41e+7, train_loss_step=0.147, val_loss=0.145, train_loss_epoch=0.144]


(RayTrainWorker pid=29191) `Trainer.fit` stopped: `max_epochs=20` reached.
2025-03-11 23:43:30,298	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/ictstr01/home/aih/serra.korkmaz/projects/saturn/hopt/hpopt_braf_zinc/ray_results/TorchTrainer_2025-03-11_23-38-37' in 0.0067s.
2025-03-11 23:43:30,311	ERROR tune.py:1037 -- Trials did not complete: [TorchTrainer_62f3f280]
2025-03-11 23:43:30,311	INFO tune.py:1041 -- Total run time: 292.85 seconds (292.77 seconds for the tuning loop).


In [9]:
results

ResultGrid<[
  Result(
    error='RayTaskError(ActorDiedError)',
    metrics={},
    path='/ictstr01/home/aih/serra.korkmaz/projects/saturn/hopt/hpopt_braf_zinc/ray_results/TorchTrainer_2025-03-11_23-38-37/62f3f280',
    filesystem='local',
    checkpoint=None
  ),
  Result(
    metrics={'train_loss': 0.14361390471458435, 'train_loss_step': 0.147378072142601, 'val/rmse': 0.38072335720062256, 'val/mae': 0.2952577471733093, 'val_loss': 0.1449502855539322, 'train_loss_epoch': 0.14361390471458435, 'epoch': 19, 'step': 560},
    path='/ictstr01/home/aih/serra.korkmaz/projects/saturn/hopt/hpopt_braf_zinc/ray_results/TorchTrainer_2025-03-11_23-38-37/1f3ddaf5',
    filesystem='local',
    checkpoint=Checkpoint(filesystem=local, path=/ictstr01/home/aih/serra.korkmaz/projects/saturn/hopt/hpopt_braf_zinc/ray_results/TorchTrainer_2025-03-11_23-38-37/1f3ddaf5/checkpoint_000019)
  )
]>

In [10]:
result_df = results.get_dataframe()
result_df

,train_loss,train_loss_step,val/rmse,val/mae,val_loss,train_loss_epoch,epoch,step,timestamp,checkpoint_dir_name,...,pid,hostname,node_ip,time_since_restore,iterations_since_restore,config/train_loop_config/depth,config/train_loop_config/ffn_hidden_dim,config/train_loop_config/ffn_num_layers,config/train_loop_config/message_hidden_dim,logdir
0,0.143614,0.147378,0.380723,0.295258,0.14495,0.143614,19,560,1741733007,checkpoint_000019,...,28562,cpusrv10.scidom.de,10.233.0.20,255.221482,20,2,2200,2,400,1f3ddaf5


In [11]:
# best configuration
best_result = results.get_best_result()
best_config = best_result.config
best_config['train_loop_config']

{'depth': 2,
 'ffn_hidden_dim': 2200,
 'ffn_num_layers': 2,
 'message_hidden_dim': 400}

In [12]:
# best model checkpoint path
best_result = results.get_best_result()
best_checkpoint_path = Path(best_result.checkpoint.path) / "checkpoint.ckpt"
print(f"Best model checkpoint path: {best_checkpoint_path}")

Best model checkpoint path: /ictstr01/home/aih/serra.korkmaz/projects/saturn/hopt/hpopt_braf_zinc/ray_results/TorchTrainer_2025-03-11_23-38-37/1f3ddaf5/checkpoint_000019/checkpoint.ckpt


In [13]:
mpnn = models.MPNN.load_from_checkpoint(best_checkpoint_path)
mpnn

MPNN(
  (message_passing): BondMessagePassing(
    (W_i): Linear(in_features=86, out_features=400, bias=False)
    (W_h): Linear(in_features=400, out_features=400, bias=False)
    (W_o): Linear(in_features=472, out_features=400, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
    (tau): ReLU()
    (V_d_transform): Identity()
    (graph_transform): Identity()
  )
  (agg): MeanAggregation()
  (bn): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (predictor): RegressionFFN(
    (ffn): MLP(
      (0): Sequential(
        (0): Linear(in_features=400, out_features=2200, bias=True)
      )
      (1): Sequential(
        (0): ReLU()
        (1): Dropout(p=0.0, inplace=False)
        (2): Linear(in_features=2200, out_features=2200, bias=True)
      )
      (2): Sequential(
        (0): ReLU()
        (1): Dropout(p=0.0, inplace=False)
        (2): Linear(in_features=2200, out_features=1, bias=True)
      )
    )
    (criterion): MSE(task_weights=[[1.0]

In [14]:
import torch
test_loader = data.build_dataloader(test_dset, shuffle=False)
with torch.inference_mode():
    trainer = pl.Trainer(
        logger=None,
        enable_progress_bar=True,
        accelerator="cpu",
        devices=1
    )
    test_preds = trainer.predict(mpnn, test_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
SLURM auto-requeueing enabled. Setting signal handlers.
/home/aih/serra.korkmaz/miniconda3/envs/saturn/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


Predicting DataLoader 0: 100%|██████████| 4/4 [00:00<00:00, 12.52it/s]


In [15]:
import numpy as np
test_preds = np.concatenate(test_preds, axis=0)
df_test['preds'] = test_preds
df_test

,smiles,value,preds
0,O=C(/C=C/c1cccnc1)Nc1ccc(C(=O)Nc2nccs2)cc1,9.2,8.836904
1,Cc1ccc(-n2nnnc2N2CCN(c3ccc(O)cc3)CC2)cc1,9.7,8.810091
2,CCCN(CCC)C(=O)c1ccc2c(c1)[C@H]1C=CC[C@H]1[C@H]...,8.7,7.913101
3,O=C(OC[C@@H](O)CO)c1ccccc1Nc1cc[nH+]c2cc(Cl)ccc12,8.3,8.283792
4,Cn1cc(CCC(=O)N2CCC[C@H](n3cncn3)C2)cn1,7.1,7.189588
...,...,...,...
191,CN1CCO[C@H](CN(C)c2cccc(/C(N)=N/O)c2)C1,8.3,7.679236
192,Cc1ccc(C(=O)CCC(=O)Nc2cc(Cl)c(O)c(Cl)c2)cc1,8.9,8.762691
193,O=C1CCCN1CC[NH+]1CCC[C@H](c2ccccc2)CC1,7.7,8.115525
194,COC(=O)c1c(N)c2cc(F)ccc2n1C,6.9,6.939610


In [16]:
from sklearn.metrics import r2_score, mean_squared_error

# Get true values (ground truth) and predictions
y_true = df_test['value'].values  # True target values
y_pred = df_test['preds'].values  # Predicted values

# Calculate metrics
r2 = r2_score(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)

print(f"R² Score: {r2:.4f}")
print(f"MSE: {mse:.4f}")

R² Score: 0.6602
MSE: 0.4056
